In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

### Load the experiment and corresponding libraries

In [2]:
project_path = os.path.join(Path.home(), "projects")
sys.path.append(project_path)  # enable import of "bias_transfer_recipes"
os.chdir(os.path.join(project_path,"bias_transfer_recipes/work/"))

In [4]:
from bias_transfer_recipes.bias_transfer_recipes.main import load_experiment
experiment = load_experiment(recipe="_2021_01_28_neurips", 
                             experiment="baselines", 
                             schema="bias_transfer_test",  # schema name will automatically be generated from recipe and experiment names 
                             base_dir=os.path.join(project_path,"bias_transfer_recipes/bias_transfer_recipes/"),
                             import_prefix="bias_transfer_recipes.bias_transfer_recipes.", 
                             dev_mode=True  
                            )

In [5]:
from bias_transfer.tables.transfer import *
from bias_transfer.tables.nnfabrik import *
schema

Connecting anix@datajoint-db.mlcloud.uni-tuebingen.de:3306


Schema `bias_transfer_test`

### Run experiments

In [10]:
from bias_transfer_recipes.bias_transfer_recipes.main import fill_tables, run_all_experiments

In [11]:
fill_tables(experiment.transfer_experiments)  

In [12]:
run_all_experiments(experiment.transfer_experiments)

  0%|          | 0/3 [00:00<?, ?it/s]

Loading dataset: MNIST-IB
Model with 135062 parameters.
==> Starting model FashionMNIST-IB clean
Validation Epoch 0: 100%|██████████| 47/47 [00:03<00:00, 15.29it/s, img_classification accuracy =74.5833, img_classification loss =0.6807, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:04<00:00, 18.33it/s, img_classification accuracy =73.5200, img_classification loss =0.7005]


 33%|███▎      | 1/3 [00:41<01:23, 41.84s/it]

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: MNIST-IB
Model with 134954 parameters.
==> Starting model FashionMNIST-IB noise
Validation Epoch 0: 100%|██████████| 47/47 [00:02<00:00, 17.45it/s, img_classification accuracy =71.9000, img_classification loss =0.7506, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:03<00:00, 21.40it/s, img_classification accuracy =71.1200, img_classification loss =0.7643]


 67%|██████▋   | 2/3 [01:15<00:39, 39.32s/it]

...

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: MNIST-IB
Model with 134954 parameters.
Resetting all parameters
==> Starting model FashionMNIST Transfer (L2-SP) clean
Validation Epoch 0: 100%|██████████| 47/47 [00:02<00:00, 20.13it/s, img_classification P-Dist =0.0000, img_classification accuracy =71.5167, img_classification loss =0.7719, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:03<00:00, 20.04it/s, img_classification P-Dist =0.0000, img_classification accuracy =70.6500, img_classification loss =0.7947]


 17%|█▋        | 2/12 [01:07<05:37, 33.78s/it]

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: MNIST-IB
Model with 511410 parameters.
==> Starting model FashionMNIST Transfer (Freeze) clean
Training Epoch 1:  40%|████      | 169/422 [00:09<00:13, 18.43it/s, img_classification accuracy =71.0568, img_classification loss =0.7730]

 17%|█▋        | 2/12 [01:21<06:46, 40.67s/it]

KeyboardInterrupt: 

In [21]:
%debug

> /usr/local/lib/python3.8/dist-packages/torch/nn/functional.py(1612)linear()
   1610         ret = torch.addmm(bias, input, weight.t())
   1611     else:
-> 1612         output = input.matmul(weight.t())
   1613         if bias is not None:
   1614             output += bias



ipdb>  u


> /usr/local/lib/python3.8/dist-packages/torch/nn/modules/linear.py(87)forward()
     85 
     86     def forward(self, input):
---> 87         return F.linear(input, self.weight, self.bias)
     88 
     89     def extra_repr(self):



ipdb>  u


> /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(550)__call__()
    548             result = self._slow_forward(*input, **kwargs)
    549         else:
--> 550             result = self.forward(*input, **kwargs)
    551         for hook in self._forward_hooks.values():
    552             hook_result = hook(self, input, result)



ipdb>  u


> /home/anix/projects/bias_transfer/bias_transfer/trainer/main_loop_modules/representation_matching.py(95)post_forward()
     93             rep_2 = extra_outputs[rep][self.batch_size :]
     94             if self.extra_layer:
---> 95                 rep_1 = self.lin2(F.relu(self.lin1(rep_1)))
     96                 rep_2 = self.lin2(F.relu(self.lin1(rep_2)))
     97             # Compute the loss:



ipdb>  rep_1.shape


torch.Size([128, 256, 16, 16])


ipdb>  q


In [44]:
# dElete all jobs in error state:
if schema.database != "nnfabrik_core": 
    (schema.jobs & "status='error'").delete()

In [48]:
# drop the entire schema: THIS IS WILL DELETE EVERYTHING!
if schema.database != "nnfabrik_core": 
    schema.drop()

Proceed to delete entire schema `bias_transfer_test`? [yes, No]:  yes


In [46]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch

experiment = "noisy_test"
run = "next run"
state = torch.load(Checkpoint().fetch("state",as_dict=True)[0]["state"])
tracker = state["tracker"]


In [47]:
def write_to_tensor_board(experiment, run, tracker):
    writer = SummaryWriter()
    def write_objective(obj_names, obj):
        if isinstance(obj,dict):
            for obj_name, sub_obj in obj.items():
                if obj_name == "normalization":
                    continue
                write_objective(obj_names+[obj_name],sub_obj)
        else:
            obj_val = tracker.get_objective(*obj_names)
            for i, val in enumerate(obj_val):
                writer.add_scalar('/'.join([run]+obj_names), val, i)
            
    write_objective([],tracker.objectives)
write_to_tensor_board(experiment, run, tracker)

/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm


In [33]:
tracker.objectives

{'LR': 0,
 'Training': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Validation': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Test': {'NoiseAdvTraining': {'loss': 0, 'normalization': 0}}}

In [42]:
tracker.log

{'LR': [0.0003, 0.0003, 0.0003, 0],
 'Training': {'img_classification': {'loss': [312.89678955078125,
    477.7629699707031,
    316.83843994140625,
    0],
   'accuracy': [1900, 1900, 1100, 0],
   'normalization': [128, 128, 128, 0]},
  'NoiseAdvTraining': {'loss': [3.6585111916065216,
    5.0233617424964905,
    1.6073499619960785,
    0],
   'normalization': [60, 60, 60, 0]}},
 'Validation': {'img_classification': {'loss': [297.705322265625,
    298.799072265625,
    294.47955322265625,
    292.99310302734375],
   'accuracy': [2500, 1200, 3400, 2800],
   'normalization': [128, 128, 128, 128]},
  'NoiseAdvTraining': {'loss': [1.491750106215477,
    1.491749994456768,
    1.491749882698059,
    1.491750106215477],
   'normalization': [60, 60, 60, 60]}},
 'Test': {'NoiseAdvTraining': {'loss': [0, 0, 0, 0],
   'normalization': [0, 0, 0, 0]}}}

In [36]:
tracker.get_objective("LR")
tracker.get_objective("Training","img_classification","accuracy")

array([14.84375, 14.84375,  8.59375,      nan])

In [5]:
import torch
import numpy as np

In [6]:
from torchvision.datasets import SVHN
from torchvision import transforms

In [7]:
transform = transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
        ])
train = SVHN(
            root="/work/",
            split="train",
            download=True,
            transform=transform,
        )

Using downloaded and verified file: /work/train_32x32.mat


In [8]:
train_set = torch.utils.data.DataLoader(
        train,
        batch_size=64,
        shuffle=False,
    )
next(iter(train_set))[0].shape

torch.Size([64, 3, 32, 32])

In [65]:
mean = 0.
std = 0.
nb_samples = 0.
data_ = []
for data, _ in train_set:
    data_.append(data)
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
dataset = torch.cat(data_)

In [66]:
print(mean,std)

tensor([0.4377, 0.4438, 0.4728]) tensor([0.1201, 0.1231, 0.1052])


In [67]:
std_ = dataset.std([0, 2, 3])
mean_ = dataset.mean([0, 2, 3])

In [68]:
std_

tensor([0.1980, 0.2010, 0.1970])

In [69]:
mean_

tensor([0.4377, 0.4438, 0.4728])

In [70]:
dataset.shape

torch.Size([73257, 3, 32, 32])

In [5]:
import datajoint as dj

In [7]:
x=list(range(10000))

In [8]:
import pickle as pkl

In [11]:
with open("x_out.pkl", "wb") as f:
    pkl.dump(x,f)

In [14]:
@schema
class DataStorage(dj.Manual):
    storage = "minio"
    
    @property
    def definition(self):
        definition = """
        # Contains the data generated by the transfer step, stored externally.
        id: varchar(128)
        ---
        data:            attach@{storage}
        """.format(
            storage=self.storage
        )
        return definition

In [15]:
DataStorage()

id,data


In [16]:
DataStorage.insert1({"id":"test","data":"./x_out.pkl"})

In [18]:
DataStorage()

id,data
test,=BLOB=


In [22]:
out = DataStorage.fetch1()

In [23]:
out

{'id': 'test', 'data': 'x_out.pkl'}